[Slides](./Networking.slides.html)

# Networking

Python has support for both high-level network protocols (HTTP/HTTPS/FTP/SMTP) as well as low-level protocols (sockets).

In this module you'll learn

- How to use `urllib`/`urllib2` to fetch web resources
- How to send email using the `smtplib` and `email` modules
- How to do basic socket programming in Python
- How you might build a REST API client in Python

## Fetching web resources with urllib/urllib2

The `urllib` and `urllib2` standard library modules allow you to open a URL (almost) just like a file. For example:

In [1]:
import urllib
import urllib2

In [2]:
resp = urllib2.urlopen('http://www.bloomberg.net')
html = resp.read()
len(html)

57228

You can also POST data to a URL using a `Request` and get the response:

In [3]:
url = 'http://localhost:8000/cgi-bin/print_request.py'
values = {'name' : 'Rick Copeland',
          'state' : 'Georgia' }

data = urllib.urlencode(values)
req = urllib2.Request(url, data)
response = urllib2.urlopen(req)
print response.read()

<TITLE>CGI script output</TITLE>
<h1>What you submitted</h1>
<table><tbody>
<tr><th>state</th><td>Georgia</td></tr>
<tr><th>name</th><td>Rick Copeland</td></tr>
</tbody></table>
<a href="/">Return to index</a>



In [4]:
print data

state=Georgia&name=Rick+Copeland


In [5]:
url = 'http://localhost:8000/cgi-bin/print_json.py'
values = {'name' : 'Rick Copeland',
          'state' : 'Georgia' }

data = urllib.urlencode(values)
req = urllib2.Request(url, data)
response = urllib2.urlopen(req)
print response.read()

{"state": "Georgia", "HTTP_METHOD": "POST", "name": "Rick Copeland"}



You can also pass query parameters to a request using `urlencode`:

In [6]:
encoded_values = urllib.urlencode(values)
encoded_values

'state=Georgia&name=Rick+Copeland'

In [8]:
print urllib2.urlopen(url + '?' + encoded_values).read()

{"state": "Georgia", "HTTP_METHOD": "GET", "name": "Rick Copeland"}



## Sending email with `smtplib`

It's frequently useful to send emails from long-running scripts; Python makes this easy with `smtplib`:

In [10]:
import smtplib

Most of the time, you'll be connecting to either your ISP's SMTP server, or some other server with authentication. In order to avoid exposing my password and such, I'll just use a local proxy server to print some debugging information:

(You can run the proxy with `python -m smtpd -nd`)

In [11]:
conn = smtplib.SMTP('localhost', 8025)
conn.set_debuglevel(1)
# conn.login(username, password)
addr_from = 'rick@arborian.com'
addr_to = 'rick446@usa.net'
msg = '''Subject: This is a test
From: <{}>
To: <{}>

Test message
'''.format(addr_from, addr_to)
conn.sendmail(addr_from, [addr_to], msg)

send: 'ehlo celeborn.local\r\n'
reply: '502 Error: command "EHLO" not implemented\r\n'
reply: retcode (502); Msg: Error: command "EHLO" not implemented
send: 'helo celeborn.local\r\n'
reply: '250 celeborn.local\r\n'
reply: retcode (250); Msg: celeborn.local
send: 'mail FROM:<rick@arborian.com>\r\n'
reply: '250 Ok\r\n'
reply: retcode (250); Msg: Ok
send: 'rcpt TO:<rick446@usa.net>\r\n'
reply: '250 Ok\r\n'
reply: retcode (250); Msg: Ok
send: 'data\r\n'
reply: '354 End data with <CR><LF>.<CR><LF>\r\n'
reply: retcode (354); Msg: End data with <CR><LF>.<CR><LF>
data: (354, 'End data with <CR><LF>.<CR><LF>')
send: 'Subject: This is a test\r\nFrom: <rick@arborian.com>\r\nTo: <rick446@usa.net>\r\n\r\nTest message\r\n.\r\n'
reply: '250 Ok\r\n'
reply: retcode (250); Msg: Ok
data: (250, 'Ok')


{}

If you're going to be putting together more complicated emails, the `email` module is extremely useful (for multipart messages, attachments, etc.):

In [12]:
from email.mime.text import MIMEText

In [13]:
msg = MIMEText('This is a text message')
msg['Subject'] = 'This is a test message'
msg['From'] = addr_from
msg['To'] = addr_to
conn.sendmail(addr_from, [addr_to], msg.as_string())

send: 'mail FROM:<rick@arborian.com>\r\n'
reply: '250 Ok\r\n'
reply: retcode (250); Msg: Ok
send: 'rcpt TO:<rick446@usa.net>\r\n'
reply: '250 Ok\r\n'
reply: retcode (250); Msg: Ok
send: 'data\r\n'
reply: '354 End data with <CR><LF>.<CR><LF>\r\n'
reply: retcode (354); Msg: End data with <CR><LF>.<CR><LF>
data: (354, 'End data with <CR><LF>.<CR><LF>')
send: 'Content-Type: text/plain; charset="us-ascii"\r\nMIME-Version: 1.0\r\nContent-Transfer-Encoding: 7bit\r\nSubject: This is a test message\r\nFrom: rick@arborian.com\r\nTo: rick446@usa.net\r\n\r\nThis is a text message\r\n.\r\n'
reply: '250 Ok\r\n'
reply: retcode (250); Msg: Ok
data: (250, 'Ok')


{}

In [14]:
conn.quit()

send: 'quit\r\n'
reply: '221 Bye\r\n'
reply: retcode (221); Msg: Bye


(221, 'Bye')

# Socket programmming

If you want to use a protocol that Python doesn't support natively (or just want to use your own protocol), you can always use the lower-level `socket` module in the standard library:

In [15]:
import socket

In [16]:
# sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock = socket.socket()

In [17]:
sock.connect(('www.bloomberg.com', 80))
http_req = '''GET / HTTP/1.1
Host: www.bloomberg.com
User-Agent: Intermediate-Python/1.0
Accept: */*

'''
sock.sendall(http_req)
response = sock.recv(1024)
sock.close()
print len(response)

1024


In [18]:
print response

HTTP/1.1 200 OK
Server: Apache
Content-Type: text/html; charset=utf-8
ETag: W/"8tSnUkosAJHFkipeYRdAvw=="
Cache-Control: public, max-age=62
Expires: Thu, 28 Apr 2016 14:05:05 GMT
Date: Thu, 28 Apr 2016 14:04:03 GMT
Transfer-Encoding:  chunked
Connection: keep-alive
Connection: Transfer-Encoding
Set-Cookie: ak_rg=US; expires=Thu, 05-May-2016 14:04:03 GMT; path=/
Set-Cookie: ak_co=US; expires=Thu, 05-May-2016 14:04:03 GMT; path=/

00006000
<!DOCTYPE html>
<html xmlns:og="http://ogp.me/ns#" data-view-uid="0"><head>
<base href='http://www.bloomberg.com/'> <meta charset="utf-8"> <title>Bloomberg.com</title> <meta http-equiv="X-UA-Compatible" content="IE=11,10,9"> <script type="text/javascript"> (function(windough, doc) { if ( doc.documentMode === 9 || doc.documentMode === 10 || (!windough.CSSMediaRule && !windough.matchMedia && !windough.msMatchMedia) ) { doc.documentElement.className += " bb-unsupported-browser"; windough["bb-unsupported-browser"] = true;function message() { var css = ".bb-

You can, of course, also build a socket server:

In [19]:
sock_srv = socket.socket()
sock_srv.bind(('127.0.0.1', 8042))
sock_srv.listen(1)
sock_conn, cli_addr = sock_srv.accept()
print 'Got a connection from', cli_addr
msg = sock_conn.recv(1024)
print msg
sock_conn.sendall(msg)
sock_conn.close()

Got a connection from ('127.0.0.1', 53209)
Hello, there



Normally, you'd do something more like the following:

In [20]:
import threading

def handle_echo_connection(sock, addr):
    print 'Handling connection from', addr
    while True:
        msg = sock.recv(1024)
        if not msg:
            break
        print 'Got', msg.rstrip()
        sock.sendall(msg)
    sock.close()
    print 'Exiting handler'
        
sock_srv = socket.socket()
sock_srv.bind(('127.0.0.1', 8043))
sock_srv.listen(1)

while True:
    sock_conn, cli_addr = sock_srv.accept()
    thd = threading.Thread(target=handle_echo_connection, args=(sock_conn, cli_addr))
    thd.setDaemon(True)
    thd.start()
    break   # normally we'd keep looping

print 'Closing server socket'
sock_srv.close()

Handling connection fromClosing server socket
 ('127.0.0.1', 53215)
Got Hello, there
Exiting handler


## Building a simple JSON REST API client

We're going to use a simple in-memory key/value store with a few endpoints:

 - GET / - return a list of collection names
 - GET /{cname} - return a list of ids for documents in a collection
 - POST /{cname} - insert a document (with auto-generated id) in a collection
 - DELETE /{cname} - drop a collection
 - GET /{cname}/{id} - return a document
 - PUT /{cname}/{id} - insert/replace a document
 - DELETE /{cname}/{id} - delete a document

In [21]:
import json
import httplib

class Database(object):
    
    def __init__(self, url):
        self._url = url
        
    def collections(self):
        resp = json.load(urllib2.urlopen(self._url))
        return resp

In [22]:
db = Database('http://localhost:5000')
db.collections()

{u'collections': []}

Unfortunately, `urllib2` requests are a little broken when it comes to HTTP methods, so we'll have to subclass them:

In [23]:
class PUT(urllib2.Request):
    def get_method(self):
        return 'PUT'

class DELETE(urllib2.Request):
    def get_method(self):
        return 'DELETE'

In [24]:
class Database(object):
    
    def __init__(self, url):
        self._url = url
        
    def collections(self):
        resp = json.load(urllib2.urlopen(self._url))
        return resp
    
    def _href_collection(self, cname):
        return '%s/%s' % (self._url, cname)
    
    def __getitem__(self, cname):
        return Collection(self._href_collection(cname))
    
    def __delitem__(self, cname):
        req = DELETE(self._href_collection(cname))
        return json.load(urllib2.urlopen(req))

class Collection(object):
    
    def __init__(self, url):
        self._url = url
        
    def documents(self):
        resp = json.load(urllib2.urlopen(self._url))
        return resp
    
    def drop(self):
        req = DELETE(self._url)
        return json.load(urllib2.urlopen(req))
    
    def insert(self, doc):
        return json.load(urllib2.urlopen(
            self._url, json.dumps(doc)))
    
    def _href_doc(self, id):
        return '%s/%s' % (self._url, id)
    
    def __getitem__(self, id):
        return json.load(urllib2.urlopen(self._href_doc(id)))
    
    def __setitem__(self, id, value):
        req = PUT(
            self._href_doc(id), 
            json.dumps(value), 
            {'Content-Type': 'application/json'})
        return json.load(urllib2.urlopen(req))
    
    def __delitem__(self, id):
        req = DELETE(self._href_doc(id))
        response = urllib2.urlopen(req)
        return json.load(response)
    
    

In [39]:
db = Database('http://localhost:5000')
coll = db['foo']
coll['0'] = {'name': 'Foo'}
db.collections()

{u'collections': [{u'cname': u'foo', u'href': u'/foo'}]}

In [26]:
coll.documents()

{u'docs': [{u'href': u'/foo/0', u'id': u'0'}]}

In [28]:
coll.insert({'name': 'Bar'})

{u'href': u'/foo/0jA8fw3C4doR/DHW', u'id': u'0jA8fw3C4doR/DHW'}

In [29]:
docs = coll.documents()
docs = dict((d['id'], d['href']) for d in docs['docs'])
docs

{u'0': u'/foo/0', u'0jA8fw3C4doR/DHW': u'/foo/0jA8fw3C4doR/DHW'}

In [30]:
coll[0]

{u'name': u'Foo'}

In [32]:
del db['foo']['0']

In [33]:
coll.documents()

{u'docs': [{u'href': u'/foo/0jA8fw3C4doR/DHW', u'id': u'0jA8fw3C4doR/DHW'}]}

In [34]:
coll.drop()

{}

In [35]:
db.collections()

{u'collections': []}

Python Requests library (3rd party) simplifies many of these tasks.

http://docs.python-requests.org/en/master/